In [ ]:
import pathlib
import torch
import numpy as np
import matplotlib.pyplot as plt

# 2. Predicting Storm Surge in Venice

### Goal

Predict water level of the next 24h


![stations](MSLP_stations.png)


### Training Data

* Simulated mean sea level pressure (MSLP)
* Measured wind speed and direction
* Measured water level

One FNN example consists of:

1. The simulated MSLP values previous 24h + current MSLP + next 24h sampled every 3h

```
14 stations * (8 + 8 + 1) = 238
```

2. Wind speed and direction at PI station 10h (1 hourly) + current values

```    
2 measurements * (10 + 1) = 22
```

3. Water level (surge only) previous 50h (1 hourly) + current value

```
1 measurement * (50 + 1) = 51
```

Which makes a total of 311 features per example.


### PyTorch Concepts

* Datasets
* Dataloaders
* Samplers
* Networks
* Training

# Datasets

In [ ]:
import h5py
from torch.utils.data import Dataset

class HDF5Dataset(Dataset):
    """Read HDF5 files from a directory"""

    def __init__(self, directory):
        self.directory = pathlib.Path(directory)
        self.files = list(self.directory.glob("*.hdf5"))

    def __getitem__(self, index):
        with h5py.File(self.files[index], "r") as src:
            inputs = src["inputs"][:]
            labels = src["labels"][:]
        return inputs, labels

    def __len__(self):
        return len(self.files)

In [ ]:
dataset = HDF5Dataset('../../exchange/venice/all_data/devel_data/test')
for _ in range(5):
    idx = np.random.randint(low=0, high=len(dataset))
    inputs, labels = dataset[idx]
    plt.plot(labels)
plt.show()
print(f"Inputs shape: {inputs.shape}")
print(f"Labels shape: {labels.shape}")

# Dataloaders

In [ ]:
from torch.utils.data import DataLoader

loader = iter(DataLoader(dataset, batch_size=5, shuffle=True, num_workers=1))
inputs, labels = next(loader)
for y in labels:
    plt.plot(y.numpy())
plt.show()

# Samplers

A very convenient way of controlling the statistics of your mini-batches

In [ ]:
from torch.utils.data import Sampler

class ExtremeRandomSampler(Sampler):
    """Sample a random extreme event with a given probability"""

    def __init__(self, dataset, num_samples=None, extreme_height=0.4, extreme_probability=0.5):
        self.dataset = dataset
        self.num_samples = len(dataset) if num_samples is None else num_samples
        self.extreme_height = extreme_height
        self.extreme_probability = extreme_probability
        self.prob = [extreme_probability, 1 - extreme_probability]
        self.length = len(dataset)

    def __iter__(self):
        indices = []
        for _ in range(self.num_samples):
            extreme = np.random.choice([False, True], p=self.prob)
            idx = np.random.randint(low=0, high=self.length)

            while not extreme:
                _, labels = self.dataset[idx]
                if np.any(labels > self.extreme_height):
                    extreme = True
                else:
                    idx = np.random.randint(low=0, high=self.length)
            indices.append(idx)
        return iter(indices)

    def __len__(self):
        return self.length

In [ ]:
from torch.utils.data import DataLoader, RandomSampler

sampler = ExtremeRandomSampler(
    dataset,
    num_samples=5,
    extreme_probability=1.,
    extreme_height=0.4)

loader = iter(DataLoader(dataset, batch_size=5, sampler=sampler))
inputs, labels = next(loader)
for y in labels:
    plt.plot(y.numpy())

length = y.size(0)
plt.plot(np.arange(length), np.zeros(length)+0.4, '-.', color='black')
plt.show()

# Neural Networks

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
BATCH_SIZE = 1000
INPUT_SIZE = 311
OUTPUT_SIZE = 24
HIDDEN_SIZE = 1024

LEARNING_RATE = 0.001
N_EPOCHS = 2

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_ds = HDF5Dataset('../../exchange/venice/all_data/devel_data/test')
val_ds = HDF5Dataset('../../exchange/venice/all_data/devel_data/test')

# with or without sampler
sampler = ExtremeRandomSampler(
    dataset, extreme_height=0.4, extreme_probability=0.01)

train_dl = DataLoader(
    train_ds, batch_size=BATCH_SIZE,
    shuffle=True
    # sampler=sampler
)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

model = FNN(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

### Train Loop - Manually

A higher level tool for training loops is for example [ignite](https://pytorch.org/ignite)

In [ ]:
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch+1}/{N_EPOCHS}")
    
    # Train
    model.train()  # IMPORTANT: some models have train/eval specific behavior
    
    running_loss, correct = 0.0, 0
    for X, y in train_dl:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        # with torch.set_grad_enabled(True):
        y_ = model(X)
        loss = criterion(y_, y)

        loss.backward()
        optimizer.step()

        # Statistics
        print(f"    batch loss: {loss.item():0.3f}")
        running_loss += loss.item() * X.shape[0]

    print(f"  Train Loss: {running_loss / len(train_dl.dataset)}")


    # Eval
    model.eval()  # IMPORTANT

    running_loss, correct = 0.0, 0
    with torch.no_grad():  # IMPORTANT
        for X, y in val_dl:
            X, y = X.to(device), y.to(device)

            y_ = model(X)

            loss = criterion(y_, y)
            running_loss += loss.item() * X.shape[0]

    print(f"  Valid Loss: {running_loss / len(val_dl.dataset)}")
    print()

## Results!

![performance](https://raw.githubusercontent.com/DHI/venice-predict/master/images/performance.png?token=AcLu2dIwONN3tLNTNcn3iB5JcnY4McW1ks5cStxewA%3D%3D)

# Thank you!

Some further links:

* Talk by [Stefan Sotte](https://github.com/sotte/pytorch_tutorial)
* Transfer Learning [Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)
* Use all of Tensorboards visualization features via [tensorboardX](https://github.com/lanpa/tensorboardX)!
* [torchvision](https://pytorch.org/docs/stable/torchvision/index.html) for transforms, pre-trained models, etc...